### Logistic Regression

In [161]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_csv('log_data.csv')
#df.head().T #edit it out to not make the text too verbose


### Identify if the column data values are in proper data type


In [162]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

### Convert the columns that should be numerical but in string format to be numerical


In [163]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

### For missing values, decide what to do to them

In [164]:
#In this case, we made all missing values in this column to 0
df['TotalCharges'] = df['TotalCharges'].fillna(0)

### For string columns, normalize them by changing spaces to underscore and lowercasing everything

In [165]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
string_columns = list(df.dtypes[df.dtypes == 'object'].index)
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

### Transform the binary categorical variable to numerical variable

In [166]:
df.churn = (df.churn == 'yes').astype(int)

### Decide what train test validation data split you'll use, but in this case you are using just the basic form, not the K fold cross validation which is the more accurate one. 

In [167]:
from sklearn.model_selection import train_test_split
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)
y_train = df_train.churn.values
y_val = df_val.churn.values
del df_train['churn']
del df_val['churn']

### Identify if the columns have missing values

In [168]:
df_train_full.isnull().sum()


customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

### Check the Distribution of target variable

In [169]:
df_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

### Separate out the categorical variable to numerical variables

In [170]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

### Identify the unique variables in each categorical variable

In [171]:
df_train_full[categorical].nunique()


gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

### Perform 2 Feature Importance Analysis: (1) Risk to determine which value of categorical variable is more at churn, and (2) Mutual Information to rank which categorical variable is more important and Correlation to determine ranking of relationship between numerical variable to target variable


In [172]:
#RISK
from IPython.display import display
global_mean = df_train_full.churn.mean()
round(global_mean, 3)
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)


,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


In [173]:
#Mutual Information for Strength of Relationship between Categorical variable and Target Variable
from sklearn.metrics import mutual_info_score
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
display(df_mi.tail())

#Correlation Coefficient for Strength of Relationship between Numerical variable and Target Variable
df_train_full[numerical].corrwith(df_train_full.churn).to_frame('correlation')


,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923


,MI
partner,0.009968
seniorcitizen,0.009410
multiplelines,0.000857
phoneservice,0.000229
gender,0.000117


,correlation
tenure,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353


### Transform categorical variables to Numerical Using One Hot Encoding Since Machine Learning requires that input be in Numerical Form

In [174]:
from sklearn.feature_extraction import DictVectorizer
train_dict = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)
X_train.shape



(3774, 45)

### Train Logistic Regression on Full Features of Dataset

In [175]:
#Train the logistic regression model with a given data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

#Apply the trained model on the validation data
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)[:, 1]

#Determine what is the accuracy of the validation data to predicted data
churn = y_pred > 0.5
(y_val == churn).mean()


0.8016129032258065

### Model Interpretation

In [176]:
#baseline prediction
model.intercept_[0]


-0.12198842052821517

In [177]:
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

c:\Users\Glaiza\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'contract=month-to-month': 0.563,
 'contract=one_year': -0.086,
 'contract=two_year': -0.599,
 'dependents=no': -0.03,
 'dependents=yes': -0.092,
 'deviceprotection=no': 0.1,
 'deviceprotection=no_internet_service': -0.116,
 'deviceprotection=yes': -0.106,
 'gender=female': -0.027,
 'gender=male': -0.095,
 'internetservice=dsl': -0.323,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.116,
 'monthlycharges': 0.001,
 'multiplelines=no': -0.168,
 'multiplelines=no_phone_service': 0.127,
 'multiplelines=yes': -0.081,
 'onlinebackup=no': 0.136,
 'onlinebackup=no_internet_service': -0.116,
 'onlinebackup=yes': -0.142,
 'onlinesecurity=no': 0.258,
 'onlinesecurity=no_internet_service': -0.116,
 'onlinesecurity=yes': -0.264,
 'paperlessbilling=no': -0.213,
 'paperlessbilling=yes': 0.091,
 'partner=no': -0.048,
 'partner=yes': -0.074,
 'paymentmethod=bank_transfer_(automatic)': -0.027,
 'paymentmethod=credit_card_(automatic)': -0.136,
 'paymentmethod=electronic_check': 0.175,


In [178]:
#Apply the sigmoid to the final value, and that will be the predicted probability. If we just use weights, the sigmoid of those weights would be 

### Use a smaller version of the Model

In [179]:
#For choosing 
small_subset = ['contract', 'tenure', 'totalcharges']
train_dict_small = df_train[small_subset].to_dict(orient='records')
dv_small = DictVectorizer(sparse=False)
dv_small.fit(train_dict_small)
X_small_train = dv_small.transform(train_dict_small)
model_small = LogisticRegression(solver='liblinear', random_state=1)
model_small.fit(X_small_train, y_train)


val_dict_small = df_val[small_subset].to_dict(orient='records')
X_small_val = dv_small.transform(val_dict_small)
y_pred_small = model_small.predict_proba(X_small_val)[:, 1]

### Use the Model

In [180]:
# The data should be in dictionary form, just like all the preprocessing steps that we did on our raw dataset when we train the model. In this case, we just aimed directly to the final version of data
customer = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75,
}
#Convert the dictionary to a matrix
X_test = dv.transform([customer])
#Use the matrix as input to the trained model, and get the 2nd column value because it represents churn
#In this case, we only have 1 data, so the zeroth row, and get the 2nd column, so we use 1. 
model.predict_proba(X_test)[0, 1]
print(list(X_test[0]))


[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 79.85, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 41.0, 3320.75]


## EVALUATION OF THE PERFORMANCE OF THE MODEL

### Calculating Accuracy of The Complete Model (Where all Features are Used)

In [181]:
# MANUAL CALCULATION OF CLASSIFICATION ACCURACY 
y_pred = model.predict_proba(X_val)[:, 1]
churn = y_pred >= 0.5
print(f'manual calculation of accuracy: {(churn == y_val).mean()}')

# AUTOMATIC CALCULATION OF CLASSIFICATION ACCURACY
from sklearn.metrics import accuracy_score
print(f'automatic calculation of accuracy: {accuracy_score(y_val, y_pred >= 0.5)}')

#Identify the best threshold by looking at 10 different points assuming accuracy is your metric
thresholds = np.linspace(0, 1, 21)
accuracies = []
for t in thresholds:
    acc = accuracy_score(y_val, y_pred >= t)
    accuracies.append(acc)
    print('%0.2f %0.3f' % (t, acc))


manual calculation of accuracy: 0.8016129032258065
automatic calculation of accuracy: 0.8016129032258065
0.00 0.261
0.05 0.501
0.10 0.595
0.15 0.640
0.20 0.690
0.25 0.730
0.30 0.755
0.35 0.767
0.40 0.782
0.45 0.795
0.50 0.802
0.55 0.790
0.60 0.790
0.65 0.788
0.70 0.774
0.75 0.752
0.80 0.742
0.85 0.739
0.90 0.739
0.95 0.739
1.00 0.739


### Calculating Accuracy of The Small Model (Where Some Features are Used)

In [182]:
churn_small = y_pred_small >= 0.5
(churn_small == y_val).mean()
print(f'Accuracy ng small model: {accuracy_score(y_val, churn_small)}')


Accuracy ng small model: 0.7672043010752688


### Create a baseline Model to Determine How Good the Accuracy Score is

In [183]:
size_val = len(y_val)
baseline = np.repeat(False, size_val)
baseline
print(f'Accuracy ng baseline model: {accuracy_score(baseline, y_val)}')


Accuracy ng baseline model: 0.7387096774193549


### Get the Confusion Table of the Big Model

In [184]:
true_positive = ((y_pred >= 0.5) & (y_val == 1)).sum()
false_positive = ((y_pred >= 0.5) & (y_val == 0)).sum()
false_negative = ((y_pred < 0.5) & (y_val == 1)).sum()
true_negative = ((y_pred < 0.5) & (y_val == 0)).sum()

confusion_table = np.array(
     # predict neg    pos
    [[true_negative, false_positive], # actual neg
     [false_negative, true_positive]]) # actual pos

print("Eto ang percentage ng TN, TP, FP at FN")
confusion_table / confusion_table.sum()

array([[0.64623656, 0.09247312],
       [0.10591398, 0.15537634]])

### Get the Precision and Recall of the Big Model

In [185]:
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
precision, recall

(0.6268980477223427, 0.5946502057613169)

## Calculate the AUC score

An AUC of 0.9 is indicative of a reasonably good model; 0.8 is okay, 0.7 is not very performant, and 0.6 indicates quite poor performance.

In [188]:
from sklearn.metrics import roc_auc_score
print(f'AUC ng full model: {roc_auc_score(y_val, y_pred)}')
print(f'AUC ng small model: {roc_auc_score(y_val, y_pred_small)}')


AUC ng full model: 0.8363366398907399
AUC ng small model: 0.8129354083179088


In [ ]:
#AUC
